## Create custom dataset from Visual Genome

In [3]:
import sys, os
sys.path.append("../")
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()

In [15]:
import os, shutil, gzip, json
from django.core.files.uploadedfile import SimpleUploadedFile
from dvaapp.shared import handle_uploaded_file
from dvaapp import models
from dvaapp.models import TEvent
from dvaapp.tasks import extract_frames, export_video_by_id
from collections import defaultdict

In [5]:
dirname = "/Volumes/Samsung_T1/visual_genome/"
outdirname = "/Volumes/Samsung_T1/"
object_name = 'license plate'
try:
    shutil.rmtree('{}/temp'.format(outdirname))            
except:
    pass
try:
    os.mkdir('{}/temp'.format(outdirname))            
except:
    pass

In [10]:
data = defaultdict(list)
with gzip.open('{}/visual_genome_objects.txt.gz'.format(dirname)) as metadata:
    for line in metadata:
        entries = line.strip().split('\t')
        if object_name in entries[6:]:
            shutil.copy('{}/{}.jpg'.format(dirname,entries[1]),'{}/temp/{}.jpg'.format(outdirname,entries[1]))
            data[entries[1]].append({
                'x':int(entries[2]),
                'y':int(entries[3]),
                'w':int(entries[4]),
                'h':int(entries[5]),
                'object_id':entries[0],
                'object_name':entries[6],
                'metadata_text':' '.join(entries[6:]),})
os.system('zip vg.zip -r {}/temp'.format(outdirname))

0

In [11]:
name = "visual_genome {}".format(object_name)
fname = "vg.zip"
f = SimpleUploadedFile(fname, file(fname).read(), content_type="application/zip")
v = handle_uploaded_file(f, name)
extract_frames(TEvent.objects.create(video=v).pk)

In [16]:
video = v
models.Region.objects.all().filter(video=video).delete()
buffer = []
for frame in models.Frame.objects.all().filter(video=video):
    frame_id = str(int(frame.name.split('/')[-1].split('.')[0]))
    for o in data[frame_id]:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video = v
        annotation.frame = frame
        annotation.x = o['x']
        annotation.y = o['y']
        annotation.h = o['h']
        annotation.w = o['w']
        annotation.object_name = o['object_name']
        annotation.metadata_json = json.dumps(o)
        annotation.metadata_text = o['metadata_text']
        buffer.append(annotation)
        if len(buffer) == 1000:
            models.Region.objects.bulk_create(buffer)
            print "saving"
            buffer = []
models.Region.objects.bulk_create(buffer)
export_video_by_id(TEvent.objects.create(video=v).pk)

saving
saving


'1_1496595768.dva_export.zip'